# Convert ROOT files Barfeature

In [1]:
import ROOT
import h5py
import numpy as np

Welcome to JupyROOT 6.16/00


To create the HDF5 file, it is good to know the number of events to store ahead of time. With ROOT, this requires a bit more code than it should, as you can't have multiple files opened.

In [2]:
def get_num_events(infiles):
    # List Comprehension does not work with ROOT
    num_events = 0
    for _, filename in infiles:
        tfile = ROOT.TFile.Open(filename)
        ttree = tfile.Get("evt")
        num_events += ttree.GetEntries()
    return num_events

In [3]:
def create_hdf5(infiles, outfile, nDP):
    num_bars = num_dp * 100
    num_events = get_num_events(infiles)
    chunksize = 1

    print(f"->     Writing to {outfile}")
    with h5py.File(outfile, "w") as h5file:
        flatfeatures = h5file.create_dataset(
            "flatfeatures",
            shape=(num_events, num_bars * 2),
            dtype=np.float32,
            chunks=(chunksize, num_bars * 2),
            compression="gzip",
            compression_opts=9,
        )
        fbuff = np.zeros((num_bars * 2), np.float32)

        consolidated = h5file.create_dataset(
            "consolidated", shape=(num_events, 3), dtype=np.int16
        )
        cbuff = np.zeros((3), np.int16)

        multiplicity = h5file.create_dataset("multiplicity", (num_events, 3), np.int8)
        mbuff = np.zeros((3), np.int8)

        primhitsbars = h5file.create_dataset(
            "primhitsbars",
            shape=(num_events, num_bars),
            dtype=np.int8,
            chunks=(chunksize, num_bars),
            compression="gzip",
            compression_opts=9,
        )
        pbuff = np.zeros((num_bars), np.int8)

        i = 0
        for nIn, filename in infiles:
            print(f"Reading ROOT file {filename}")
            tfile = ROOT.TFile.Open(filename)
            ttree = tfile.Get("evt")
            for event in ttree:
                # Flatfeatures Features
                fbuff.fill(0)
                for hit in event.NeulandHits:
                    bar = hit.GetPaddle() - 1
                    fbuff[2 * bar + 0] = hit.GetE()
                    fbuff[2 * bar + 1] = hit.GetT()
                flatfeatures[i] = fbuff

                # Consolidated Features
                # nHits: Number of hits
                cbuff[0] = event.NeulandHits.GetEntries()
                # nClus: Number of clusters
                cbuff[1] = event.NeulandClusters.GetEntries()
                # Edep: Total deposited (detected) energy
                cbuff[2] = round(sum([hit.GetE() for hit in event.NeulandHits]))
                consolidated[i] = cbuff

                # multiplicity
                # nPN: Number of incoming primary neutrons
                mbuff[0] = nIn
                # nPP: Number of primary neutrons with an energy deposition in NeuLAND
                mbuff[1] = event.NeulandPrimaryPoints.GetEntries()
                # nPH: Number of hits that correspond to a energy deposition of a primary neutron
                mbuff[2] = event.NeulandPrimaryHits.GetEntries()
                multiplicity[i] = mbuff

                # Primary Hits
                pbuff.fill(0)
                for phit in event.NeulandPrimaryHits:
                    bar = phit.GetPaddle() - 1
                    pbuff[bar] = 1
                primhitsbars[i] = pbuff

                i += 1

Note: Don't try parallel execution here

In [4]:
incoming_neutrons = range(1, 6)
beam_energy = 600


def create_hdf5_wrap(num_dp):
    infiles = [
        (
            neutrons,
            f"simulation/training_{beam_energy}AMeV_{num_dp}dp_{neutrons}n.digi.root",
        )
        for neutrons in incoming_neutrons
    ]
    outfile = f"data/{beam_energy}AMeV_{num_dp}dp.bars.h5"
    create_hdf5(infiles, outfile, num_dp)


for num_dp in [15, 30]:
    create_hdf5_wrap(num_dp)

->     Writing to data/training_600AMeV_15dp.barfeature.h5
Reading ROOT file simulation/training_600AMeV_15dp_1n.digi.root
Reading ROOT file simulation/training_600AMeV_15dp_2n.digi.root
Reading ROOT file simulation/training_600AMeV_15dp_3n.digi.root
Reading ROOT file simulation/training_600AMeV_15dp_4n.digi.root
Reading ROOT file simulation/training_600AMeV_15dp_5n.digi.root
->     Writing to data/validation_600AMeV_15dp.barfeature.h5
Reading ROOT file simulation/validation_600AMeV_15dp_1n.digi.root
Reading ROOT file simulation/validation_600AMeV_15dp_2n.digi.root
Reading ROOT file simulation/validation_600AMeV_15dp_3n.digi.root
Reading ROOT file simulation/validation_600AMeV_15dp_4n.digi.root
Reading ROOT file simulation/validation_600AMeV_15dp_5n.digi.root
->     Writing to data/test_600AMeV_15dp.barfeature.h5
Reading ROOT file simulation/test_600AMeV_15dp_1n.digi.root
Reading ROOT file simulation/test_600AMeV_15dp_2n.digi.root
Reading ROOT file simulation/test_600AMeV_15dp_3n.digi.